In [1]:
import os
import sys
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ

In [3]:
acq_sirius = LDAQ.dewesoft.DewesoftSiriusX()

In [4]:
acq_sirius.acquisition_name

'dewesoft'

In [5]:
acq_sirius.list_available_devices()

Name:[tid: 3410][2025-09-02 08:57:59.047] [ModuleManager] [warning] Failed to check for network settings: open: Operation not permitted [system:1 at /__w/openDAQ/openDAQ/build/__external/src/Boost/libs/asio/include/boost/asio/detail/reactive_socket_service.hpp:133 in function 'open']
 Monitor of Raptor Lake-P/U/H cAVS HDMI / DisplayPort 3 Output Connection string: miniaudio://pulseaudio/alsa_output.pci-0000_00_1f.3-platform-skl_hda_dsp_generic.HiFi__HDMI3__sink.monitor
Name: Monitor of Raptor Lake-P/U/H cAVS HDMI / DisplayPort 2 Output Connection string: miniaudio://pulseaudio/alsa_output.pci-0000_00_1f.3-platform-skl_hda_dsp_generic.HiFi__HDMI2__sink.monitor
Name: Monitor of Raptor Lake-P/U/H cAVS HDMI / DisplayPort 1 Output Connection string: miniaudio://pulseaudio/alsa_output.pci-0000_00_1f.3-platform-skl_hda_dsp_generic.HiFi__HDMI1__sink.monitor
Name: Monitor of Raptor Lake-P/U/H cAVS Speaker Connection string: miniaudio://pulseaudio/alsa_output.pci-0000_00_1f.3-platform-skl_hda_ds

In [6]:
acq_sirius.connect_to_device("daq://Dewesoft_DB24050686")

True

In [7]:
channel_settings = {
    0: {
        'Name': 'acc_X',
        'Measurement': 'IEPE', # [IEPE, Voltage]
        'Range': '10000', # [10000, 5000, 1000, 200] mV
        'HPFilter': 'AC 1Hz', # [AC 0.1Hz, AC 1Hz]
        'Excitation': 2.0, # [2, 4, 6] mA
        'Sensitivity': 100,
        'Sensitivity Unit': 'mV/g', # [mV/g, mV/(m/s^2)]
        'Unit': 'g', # [g, m/s^2]
    },
    1: {
        'Name': 'acc_Y',
        'Measurement': 'IEPE', # [IEPE, Voltage]
        'Range': '10000', # [10000, 5000, 1000, 200] mV
        'HPFilter': 'AC 1Hz', # [AC 0.1Hz, AC 1Hz]
        'Excitation': 2.0, # [2, 4, 6] mA
        'Sensitivity': 100,
        'Sensitivity Unit': 'mV/g', # [mV/g, mV/(m/s^2)]
        'Unit': 'g', # [g, m/s^2]
    },
    2: {
        'Name': 'acc_Z',
        'Measurement': 'IEPE', # [IEPE, Voltage]
        'Range': '10000', # [10000, 5000, 1000, 200] mV
        'HPFilter': 'AC 1Hz', # [AC 0.1Hz, AC 1Hz]
        'Excitation': 2.0, # [2, 4, 6] mA
        'Sensitivity': 100,
        'Sensitivity Unit': 'mV/g', # [mV/g, mV/(m/s^2)]
        'Unit': 'g', # [g, m/s^2]
    },
    3: {
        'Name': 'vol_1',
        'Measurement': 'Voltage', # [IEPE, Voltage]
        'Range': '10', # [10, 5, 1, 0.2] V
        'HPFilter': 'DC', # [DC, AC 0.1Hz, AC 1Hz] 
        'Sensitivity': 1,
        'Sensitivity Unit': 'V/V', # [arbitrary]
        'Unit': 'V', # [arbitrary]
    }
}

In [8]:
acq_sirius.configure_channels(channel_settings, sample_rate=10000)

Channels configured. You can check the current configuration with self.list_available_channels()
Sample rate was set to: 10000 Hz.


In [9]:
acq_sirius.list_available_channels()

Channel Global ID:  /openDAQDevice/Dev/Dewesoft_DB24050686/IO/1/AI1
Channel Name     :  acc_X
  Function Block Name:  Amplifier
     Measurement          : IEPE       [Voltage, IEPE] 
     Range                : 10000      [10000, 5000, 1000, 200] mV
     HPFilter             : AC 1Hz     [AC 0.1Hz, AC 1Hz] 
     Excitation           : 2.0        [2, 4, 6] mA
     ExcitationUnit       : mA         [mA] 
     SupportsLed1Control  : True       None 
Channel Global ID:  /openDAQDevice/Dev/Dewesoft_DB24050686/IO/1/AI2
Channel Name     :  acc_Y
  Function Block Name:  Amplifier
     Measurement          : IEPE       [Voltage, IEPE] 
     Range                : 10000      [10000, 5000, 1000, 200] mV
     HPFilter             : AC 1Hz     [AC 0.1Hz, AC 1Hz] 
     Excitation           : 2.0        [2, 4, 6] mA
     ExcitationUnit       : mA         [mA] 
     SupportsLed1Control  : True       None 
Channel Global ID:  /openDAQDevice/Dev/Dewesoft_DB24050686/IO/1/AI3
Channel Name     :  acc_Z
  

In [10]:
# gui
vis = LDAQ.Visualization(refresh_rate=30)
vis.add_lines(
    position=(0, 0),
    source='dewesoft',
    channels=(0,1,2),
)

In [11]:
ldaq = LDAQ.Core(acquisitions=[acq_sirius], visualization=vis)

In [12]:
ldaq.set_trigger(
    source='dewesoft',
    channel='acc_X',
    level=1000.,
    duration=1.,
    presamples=1000,
    trigger_type='abs'
)

In [13]:
ldaq.run(hotkeys=False)

closing app
closing app


In [14]:
data = ldaq.get_measurement_dict()['dewesoft']

In [15]:
import matplotlib.pyplot as plt
%matplotlib qt

In [16]:
time = data['time']
acc_X = data['data'][:,0]
acc_Y = data['data'][:,1]
acc_Z = data['data'][:,2]
vol_1 = data['data'][:,3]

plt.figure()
plt.plot(time, acc_X, label='Acc X')
plt.plot(time, acc_Y, label='Acc Y')
plt.plot(time, acc_Z, label='Acc Z')
plt.plot(time, vol_1, label='Vol 1')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
